# FSharp Samples

## Downloading Nuget Packages

In [1]:
#!fsharp
#r "nuget: XPlot.Plotly"
#r "nuget: FSharp.Data"
#r "nuget: MathNet.Numerics"
#r "nuget: Newtonsoft.Json" 

Installed package XPlot.Plotly version 3.0.1

Installed package FSharp.Data version 3.3.3

Installed package Newtonsoft.Json version 12.0.3

Installed package MathNet.Numerics version 4.12.0

## Reading a CSV File

In [1]:
#!fsharp
open FSharp.Data
open System.IO

let sp500 = CsvFile.Load(Path.Join(__SOURCE_DIRECTORY__, "./Data/SP500.csv" ))

// Print the prices in the HLOC format
sp500.Rows
|> Seq.take 5

index,Columns
0,"[ 2008-05-02, 1413.900024, 0.0032303750170896706 ]"
1,"[ 2008-05-05, 1407.48999, -0.0045438914840980615 ]"
2,"[ 2008-05-06, 1418.26001, 0.007622806087752565 ]"
3,"[ 2008-05-07, 1392.569946, -0.018279853115200417 ]"
4,"[ 2008-05-08, 1397.680054, 0.0036628357410659618 ]"


## Plots

### Plotting a Pie Chart

In [1]:
#!fsharp
open XPlot.Plotly

[ "Pizza remaining", 3; "Pizza eaten", 5 ]
|> Chart.Pie

### Plotting a Line Plot

In [1]:
#!fsharp
open XPlot.Plotly
// Source: https://gist.github.com/theprash/7b6163d55b199359cd0dc68c477af0fa 

#r "System.Net.Http"
#r "nuget: Newtonsoft.Json"

let client = new System.Net.Http.HttpClient()
let weatherJson =
    client
        .GetStringAsync("http://www.7timer.info/bin/astro.php?lon=113.2&lat=23.1&ac=0&unit=metric&output=json&tzshift=0")
        .Result

let jObj = Newtonsoft.Json.Linq.JObject.Parse weatherJson

jObj.["dataseries"]
|> Seq.map (fun x -> x.Value<float> "timepoint", x.Value<float> "cloudcover" )
|> Seq.toArray
|> Chart.Line
|> Chart.WithXTitle "Time"
|> Chart.WithYTitle "Cloud cover"

### Plotting 2 Line Plots on One Chart

In [1]:
#!fsharp
open XPlot.Plotly
// Source: https://fslab.org/XPlot/chart/plotly-line-scatter-plots.html

let trace1 =
    Scatter(
        x = [1; 2; 3; 4],
        y = [10; 15; 13; 17]
    )

let trace2 =
    Scatter(
        x = [2; 3; 4; 5],
        y = [16; 5; 11; 9]
    )

[trace1; trace2]
|> Chart.Plot
|> Chart.WithWidth 700
|> Chart.WithHeight 500

### Plotting a Histogram

In [1]:
#!fsharp
open XPlot.Plotly
open MathNet.Numerics.Distributions

let x = 
    ContinuousUniform().Samples()
    |> Seq.take 10000
    |> Seq.toArray

Histogram(x = x) 
|> Chart.Plot
|> Chart.WithWidth 700
|> Chart.WithHeight 500

## Metropolis-Hastings


### Iterative Random Walk Metropolis Hastings

In [1]:
#!fsharp
open System
open MathNet.Numerics.Distributions
open XPlot.Plotly

let gaussianMetropolisHastingsFixedProposal (numberOfIterations: int) 
                                            (mu : double) 
                                            (sigma : double) : double list =
    let mutable states : double list = [] 
    let burnin : int = int (Math.Ceiling(float (double numberOfIterations * 0.2)))

    let proposedUniform = ContinuousUniform(mu - 5.0 * sigma, mu + 5.0 * sigma)
    let zeroOneUniform  = ContinuousUniform()
    let normal          = Normal(mu, sigma)

    // Initialize the current state.
    let mutable current = proposedUniform.Sample()

    for i in 0..numberOfIterations do

        // Proposed Move
        let proposedMove = proposedUniform.Sample()

        // Acceptance Ratio
        let currentProbability = normal.Density(current)
        let proposedProbabilty = normal.Density(proposedMove) 
        let acceptanceRatio    = Math.Min(proposedProbabilty / currentProbability, 1.0)

        // Choice
        let uniformDraw = zeroOneUniform.Sample() 

        if (uniformDraw < acceptanceRatio) then current <- proposedMove

        // Append the current state 
        states <- states @ [current]

    states
    |> List.skip(burnin)

let mhRun = gaussianMetropolisHastingsFixedProposal 50000 0.0 1.0

Histogram(x = mhRun)
|> Chart.Plot
|> Chart.WithWidth 700
|> Chart.WithHeight 500

## Parsing and Parameterizing the Model with a Sampling Model 

### Parsing the Model Pattern Matching

In [1]:
#!fsharp
open System

type ParsedRandomVariable = 
    { Name             : string; 
      Conditionals     : string list; 
      Distribution     : string; 
      Parameters       : string list; 
      Observed         : string option }
type ParsedBayesianModel = ParsedRandomVariable list

// Format: RVName [|Conditionals] ~ Distribution( Parameters ) [: observed] 
// [] -> optional
let parseLineOfModel (lineInModel : string) : ParsedRandomVariable = 
 
    // Helper fn to split the string based on a variety of type of delimiters.
    // Resultant type is a list of strings to feed in for the pattern matching.
    let splitToList (toSplit : string) (delimiters : obj) : string list = 
        let split = 
            match delimiters with
            | :? string        as s   -> toSplit.Split(s, StringSplitOptions.RemoveEmptyEntries) 
            | :? array<string> as arr -> toSplit.Split(arr, StringSplitOptions.RemoveEmptyEntries) 
            | :? array<char>   as arr -> toSplit.Split(arr, StringSplitOptions.RemoveEmptyEntries) 
            | _ -> failwithf "Splitting based on delimiters failed as it is neither a string nor an array of strings: Of Type: %A - %A" (delimiters.GetType()) toSplit
        
        Array.toList split

    match splitToList lineInModel " " with
    | nameAndConditionals :: "~" :: distributionParametersObserved ->
        // Get the name and conditionals.
        let splitNameAndConditionals = splitToList nameAndConditionals "|"
        let name = splitNameAndConditionals.[0]
        let conditionals = 
            match splitNameAndConditionals with 
            | name :: conditionals -> 
                if conditionals.Length > 0 then splitToList conditionals.[0] ","
                else []
            | _ -> failwithf "Pattern not found for RV Name and Conditionals - the format is: RVName|Condtionals: %A" splitNameAndConditionals

        let extractAndGetParameters (distributionNameAndParameters : string) : string * string list = 
            let splitDistributionAndParameters = splitToList distributionNameAndParameters [| "("; ")" |]
            (splitDistributionAndParameters.[0], splitToList splitDistributionAndParameters.[1] ",")
            
        match distributionParametersObserved with 

        // Case: Without Observations. Example: theta ~ Gamma(a,b)
        | distributionNameAndParameters when distributionNameAndParameters.Length = 1 ->
            let extractedDistributionAndParameters = extractAndGetParameters distributionNameAndParameters.[0]
            { Name             = name; 
              Conditionals     = conditionals; 
              Distribution     = (fst extractedDistributionAndParameters).ToLower();
              Observed         = None; 
              Parameters       = snd extractedDistributionAndParameters; }

        // Case: With Observations. Example: Y|theta ~ Poisson(theta) : observed
        | distributionNameAndParameters :: ":" :: observed ->
            let extractedDistributionAndParameters = extractAndGetParameters distributionNameAndParameters
            { Name             = name;
              Conditionals     = conditionals; 
              Distribution     = (fst extractedDistributionAndParameters).ToLower();
              Observed         = Some observed.Head; 
              Parameters       = snd extractedDistributionAndParameters; } 

        // Case: Error.
        | _ -> failwithf "Pattern not found for the model while parsing the distribution, parameters and optionally, the observed variables: %A" distributionParametersObserved 

    | _ -> failwithf "Pattern not found for the following line in the model - please check the syntax: %A" lineInModel

let parseModel (model : string) : ParsedBayesianModel = 
    model.Split('\n') 
    |> Array.toList
    |> List.map(parseLineOfModel)

let printParsedModel (model : string) : unit = 
    let parsedModel = parseModel model
    printfn "Model: %A is represented as %A" model parsedModel

In [1]:
#!fsharp
// Print out our simple 1-Parameter Model.
let model1 = @"theta ~ Gamma(a,b)
              Y|theta ~ Poisson(theta) : observed"
printParsedModel(model1)

let model2 = @"theta ~ Beta(unit,unit)
               Y|theta ~ Binomial(n,theta) : observed"
printParsedModel(model2)

let model3 = @"theta ~ Beta(unit,unit)
               gamma ~ Gamma(a,b)
               Y|theta,gammma ~ Binomial(n,theta) : observed"
printParsedModel(model3)

Model: 

"theta ~ Gamma(a,b)
              Y|theta ~ Poisson(theta) : observed"

 is represented as 

[{ Name = "theta"
   Conditionals = []
   Distribution = "gamma"
   Parameters = ["a"; "b"]
   Observed = None }; { Name = "Y"
                        Conditionals = ["theta"]
                        Distribution = "poisson"
                        Parameters = ["theta"]
                        Observed = Some "observed" }]

Model: 

...

[{ Name = "theta"
   Conditionals = []
   Distribution = "beta"
   Parameters = ["unit"; "unit"]
   Observed = None }; { Name = "Y"
                        Conditionals = ["theta"]
                        Distribution = "binomial"
                        Parameters = ["n"; "theta"]
                        Observed = Some "observed" }]

Model: 

"theta ~ Beta(unit,unit)
               gamma ~ Gamma(a,b)
               Y|theta,gammma ~ Binomial(n,theta) : observed"

 is represented as 

[{ Name = "theta"
   Conditionals = []
   Distribution = "beta"
   Parameters = ["unit"; "unit"]
   Observed = None }; { Name = "gamma"
                        Conditionals = []
                        Distribution = "gamma"
                        Parameters = ["a"; "b"]
                        Observed = None }; { Name = "Y"
                                             Conditionals = ["theta"; "gammma"]
                                             Distribution = "binomial"
                                             Parameters = ["n"; "theta"]
                                             Observed = Some "observed" }]

### Parameterization

In [1]:
#!fsharp
#r "nuget: Newtonsoft.Json" 
#r "nuget: MathNet.Numerics"

open System
open System.Collections.Generic

open MathNet.Numerics.Distributions

open Newtonsoft.Json

type Observed = float list

type ParameterList = 
    { Observed : float list; Parameters : Dictionary<string, float> } 

let deserializeParameters (paramsAsString : string) : ParameterList = 
    JsonConvert.DeserializeObject<ParameterList>(paramsAsString)

// Parsing parameters
let parameters1 = "{Parameters : {mu0 : 0, sigma0 : 1, mu : 5, sigma : 2, lambda : 4}, observed : [4,5,2]}"
let deserializedParameters1 = deserializeParameters parameters1
printfn "%A" ( deserializedParameters1.SerializeToJson() )

let parameters2 = "{Parameters: {lambda : 2}}"
let deserializedParameters2 = deserializeParameters parameters2

// Applying the Parameters to a Distribution
let exp = Exponential deserializedParameters2.Parameters.["lambda"] 
exp.Sample()

{"Observed":[4.0,5.0,2.0],"Parameters":{"mu0":0.0,"sigma0":1.0,"mu":5.0,"sigma":2.0,"lambda":4.0}}

0.07501101293715182

### Parameterized with a Parsed Model 

In [1]:
#!fsharp
open MathNet.Numerics.Distributions

type DistributionType = 
    | Continuous
    | Discrete

type DistributionInfo = { DistributionType : DistributionType 
                          DistributionName : string; 
                          Parameters       : float list; 
                          Density          : float -> float; 
                          SamplingModel    : unit -> double;
                          
                          
                          } with
    member this.GetSample() : double =
        this.SamplingModel()
    member this.GetSamples(numberOfSamples : int) : double seq =
        seq {1..numberOfSamples}
        |> Seq.map(fun _ -> this.GetSample())

let getDistributionInfoForRV (item : ParsedRandomVariable) 
                             (parameterList : ParameterList) : DistributionInfo = 
    // I know this is ugly but this functionality assumes the user enters the 
    // parameters in the order that's expected by the MathNet Numerics Library. 
    // Grab the parameters associated with this Random Variable.

    let rvParameters = 
        item.Parameters
        |> List.filter(parameterList.Parameters.ContainsKey) 
        |> List.map(fun item -> parameterList.Parameters.[item])

    // Extract Distribution Associated with the Parsed Random Variable.
    match item.Distribution with

    // 1 Parameter Distributions 
    | "exponential" -> 
        let model = Exponential(rvParameters.[0])
        { DistributionName = item.Distribution; 
          Parameters       = rvParameters; 
          DistributionType = DistributionType.Continuous;
          Density          = model.Density;
          SamplingModel    = model.Sample; }
    | "poisson" ->
        let model = Poisson(rvParameters.[0])
        { DistributionName = item.Distribution; 
          Parameters       = rvParameters;
          DistributionType = DistributionType.Discrete;
          Density          = (fun x -> model.Probability (int x));
          SamplingModel    = (fun x -> double (model.Sample())) }
    // 2 Parameter Distributions 
    | "normal" ->
        let model = Normal(rvParameters.[0], rvParameters.[1])
        { DistributionName = item.Distribution; 
          Parameters       = rvParameters; 
          DistributionType = DistributionType.Continuous;
          Density          = model.Density;
          SamplingModel    = model.Sample; }
    | "gamma" ->
        let model = Gamma(rvParameters.[0], rvParameters.[1])
        { DistributionName = item.Distribution; 
          Parameters       = rvParameters; 
          DistributionType = DistributionType.Continuous;
          Density          = model.Density;
          SamplingModel    = model.Sample; }
    | "beta" ->
        let model = Beta(rvParameters.[0], rvParameters.[1])
        { DistributionName = item.Distribution; 
          Parameters       = rvParameters; 
          DistributionType = DistributionType.Continuous;
          Density          = model.Density;
          SamplingModel    = model.Sample; }
    // Failure Case
    | _ -> failwithf "Distribution not registered: %A" item.Distribution

let getDistributionInfoForModel(model : string) (parameterList : string) : DistributionInfo list = 
    let parsedModel   = parseModel model
    let parameterList = deserializeParameters parameterList 

    parsedModel
    |> List.map(fun x -> getDistributionInfoForRV x parameterList)

In [1]:
#!fsharp
// Exponential. 
let exponentialModel     = "x ~ Exponential(lambda)"
let exponentialParamList = "{Parameters: {lambda : 2, a : 2., b : 2.3 }, Observed : []}"
let exponentialComplied : DistributionInfo list = getDistributionInfoForModel exponentialModel exponentialParamList
printfn "5 of the Exponential Samples: %A" (exponentialComplied |> List.map(fun x -> x.GetSamples(5)))
printfn "%A" exponentialComplied

// Normal.
let normalModel     = "x ~ Normal(mu,sigma)"
let normalParamList = "{Parameters: {mu: 0, sigma : 1.}, Observed : []}"
let normalComplied  = getDistributionInfoForModel normalModel normalParamList
printfn "5 of the Normal Samples: %A" (normalComplied |> List.map(fun x -> x.GetSamples(5)))
printfn "%A" normalComplied

// Poisson.
let poissonModel          = "x ~ Poisson(theta)"
let poissonModelParamList = "{Parameters: {theta: 44}, Observed : []}"
let poissonCompiled       = getDistributionInfoForModel poissonModel poissonModelParamList
printfn "5 of the Poisson Samples: %A" (poissonCompiled |> List.map(fun x -> x.GetSamples(5)))
printfn "%A" poissonCompiled

5 of the Exponential Samples: 

[seq [0.4408509031; 0.7220424081; 0.6969660043; 0.02129114276; ...]]

[{ DistributionType = Continuous
   DistributionName = "exponential"
   Parameters = [2.0]
   Density = <fun:getDistributionInfoForRV@41-5>
   SamplingModel = <fun:getDistributionInfoForRV@42-6> }]

5 of the Normal Samples: 

[seq [3.244777035; 0.9026127367; -0.9686871519; -1.149983475; ...]]

[{ DistributionType = Continuous
   DistributionName = "normal"
   Parameters = [0.0; 1.0]
   Density = <fun:getDistributionInfoForRV@56-9>
   SamplingModel = <fun:getDistributionInfoForRV@57-10> }]

5 of the Poisson Samples: 

[seq [43.0; 54.0; 46.0; 43.0; ...]]

[{ DistributionType = Discrete
   DistributionName = "poisson"
   Parameters = [44.0]
   Density = <fun:getDistributionInfoForRV@48-7>
   SamplingModel = <fun:getDistributionInfoForRV@49-8> }]

## Constructing the Bayesian Network 

### Bayesian Node

In [1]:
#!fsharp
type BayesianNodeTypeInfo =
| Observed of float list 
| NonObserved

type BayesianNode = 
    { Name                 : string; 
      Type                 : BayesianNodeTypeInfo 
      DistributionInfo     : DistributionInfo
      ParsedRandomVariable : ParsedRandomVariable } with

    static member ConstructNode(parsedRandomVariable : ParsedRandomVariable)
                               (parameterList : ParameterList) =

        let nodeType : BayesianNodeTypeInfo =
            match parsedRandomVariable.Observed with
            | Some x -> BayesianNodeTypeInfo.Observed parameterList.Observed
            | None   -> BayesianNodeTypeInfo.NonObserved
        { Name = parsedRandomVariable.Name;
          Type = nodeType;
          DistributionInfo = getDistributionInfoForRV parsedRandomVariable parameterList;
          ParsedRandomVariable = parsedRandomVariable; }
    
    member this.GetDependents (parsedBayesianModel : ParsedBayesianModel) : ParsedRandomVariable list =
        parsedBayesianModel 
        |> List.filter(fun x -> x.Conditionals |> List.contains(this.Name))

In [1]:
#!fsharp
let lineOfModel = @"x ~ Exponential(lambda)"
let paramList   = "{Parameters: {lambda : 2, a : 2., b : 2.3 }, Observed : [1,2,3,55]}"
display(BayesianNode.ConstructNode (parseLineOfModel lineOfModel) (deserializeParameters paramList))

Name,Type,DistributionInfo,ParsedRandomVariable
x,NonObserved,"{ DistributionType = Continuous DistributionName = ""exponential"" Parameters = [2.0] SamplingModel = <fun:getDistributionInfoForRV@36> }","{ Name = ""x"" Conditionals = [] Distribution = ""exponential"" Parameters = [""lambda""] Observed = None }"


### Bayesian Network Model

In [1]:
#!fsharp
type BayesianNetworkModel =
    { Name    : string;
      Nodes   : IDictionary<string, BayesianNode>
      Network : IDictionary<string, BayesianNode list> } with

    member this.GetPosterior() =
      ()

    static member ConstructModel (name : string)
                                 (model : ParsedBayesianModel)
                                 (parameterList : ParameterList) = 

            // Construct all the modes of the model.
            let allNodes : IDictionary<string, BayesianNode> = 
                model
                |> List.map(fun m -> m.Name, BayesianNode.ConstructNode m parameterList)
                |> dict

            // From the list of all models, for each node get the dependends and append to the network.
            let network = Dictionary<string, BayesianNode list>()

            for KeyValue(name, bayesianNode) in allNodes do
              let dependents = bayesianNode.GetDependents model
              let dependentsAsBayesianNodes = 
                dependents
                |> List.map(fun d -> allNodes.[d.Name])
              network.Add(name, dependentsAsBayesianNodes)
            { Name    = name;
              Nodes   = allNodes;
              Network = network }


In [1]:
#!fsharp
let model = @"x ~ Exponential(lambda)
              y|x ~ Gamma(a,b) : observed"
let parsedModel = parseModel model
let paramList = "{Parameters: {lambda : 2, a : 2., b : 2.3 }, Observed : [1,2,3,55]}"
BayesianNetworkModel.ConstructModel "Exponential Model" parsedModel (deserializeParameters paramList)

Name,Nodes,Network
Exponential Model,"[ [x, { Name = ""x"" Type = NonObserved DistributionInfo = { DistributionType = Continuous DistributionName = ""exponential"" Parameters = [2.0] SamplingModel = <fun:getDistributionInfoForRV@36> } ParsedRandomVariable = { Name = ""x"" Conditionals = [] Distribution = ""exponential"" Parameters = [""lambda""] Observed = None } }], [y, { Name = ""y"" Type = Observed [1.0; 2.0; 3.0; 55.0] DistributionInfo = { DistributionType = Continuous DistributionName = ""gamma"" Parameters = [2.0; 2.3] SamplingModel = <fun:getDistributionInfoForRV@52-3> } ParsedRandomVariable = { Name = ""y"" Conditionals = [""x""] Distribution = ""gamma"" Parameters = [""a""; ""b""] Observed = Some ""observed"" } }] ]","[ [x, [{ Name = ""y"" Type = Observed [1.0; 2.0; 3.0; 55.0] DistributionInfo = { DistributionType = Continuous DistributionName = ""gamma"" Parameters = [2.0; 2.3] SamplingModel = <fun:getDistributionInfoForRV@52-3> } ParsedRandomVariable = { Name = ""y"" Conditionals = [""x""] Distribution = ""gamma"" Parameters = [""a""; ""b""] Observed = Some ""observed"" } }]], [y, []] ]"


## Inference

### Computing the Posterior 

In [1]:
#!fsharp
